# Notebook 6: Pedestrian Detection - A Safety-Critical Case Study

**Session 1: AI-based Perception Systems in Autonomous Vehicles**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/Autonomous-Driving_AI-Safety-and-Security/blob/main/AV_Perception_Safety_Workshop/Session_1_AI_Perception_Systems/notebooks/06_Pedestrian_Detection_Case_Study.ipynb)

**Author:** Milin Patel  
**Duration:** ~15 minutes

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- ✅ Understand why pedestrian detection is safety-critical
- ✅ Implement YOLOv8 for pedestrian detection
- ✅ Analyze failure cases and edge scenarios
- ✅ Link to ISO 26262 safety requirements
- ✅ Evaluate detection performance metrics
- ✅ Identify mitigation strategies for failures

---

## 🚨 Why Pedestrian Detection is Critical

**Pedestrians are Vulnerable Road Users (VRUs):**
- No protective shell (unlike vehicles)
- Unpredictable behavior (children, elderly, distracted)
- Small visual signature (hard to detect at distance)
- High injury/fatality risk in collisions

**Real-World Incidents:**
- **Uber ATG (2018):** Failed to detect pedestrian pushing bicycle → fatal collision
- **Tesla Autopilot (multiple):** Failures in low-light conditions
- **Waymo (2023):** Pedestrian struck in autonomous mode

**ISO 26262 Classification:**
- **ASIL-D** (highest safety level)
- Requires redundancy, validation, fail-safe mechanisms

---

## 📦 Setup and Imports

In [ ]:
import sys

# Check if running on Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔧 Running on Google Colab - Installing dependencies...\n")
    !pip install -q ultralytics opencv-python matplotlib numpy pillow requests pandas seaborn
    print("✅ Setup complete!\n")
else:
    print("💻 Running locally\n")

print("✅ Environment ready!")

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
from PIL import Image
import requests
from io import BytesIO
from ultralytics import YOLO
import time
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print("✅ All libraries imported successfully!")
print(f"NumPy version: {np.__version__}")

---

## 1️⃣ Pedestrian Detection Statistics

Let's look at real-world pedestrian accident data to understand the problem.

In [ ]:
# Pedestrian fatality data (US, approximate)
accident_conditions = pd.DataFrame({
    'Condition': ['Daylight', 'Dark (lit)', 'Dark (unlit)', 'Dawn/Dusk', 'Weather'],
    'Percentage': [25, 30, 35, 10, 15],  # Overlapping categories
    'Risk_Factor': [1.0, 3.5, 7.0, 2.5, 2.0]
})

# Pedestrian types
pedestrian_types = pd.DataFrame({
    'Type': ['Adult Walking', 'Child', 'Elderly', 'Cyclist/Scooter', 'Wheelchair', 'Pushing Object'],
    'Frequency': [60, 15, 20, 30, 2, 8],
    'Detection_Difficulty': [1.0, 1.5, 1.2, 2.0, 2.5, 3.0]  # Relative difficulty
})

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Accident conditions
axes[0, 0].bar(accident_conditions['Condition'], accident_conditions['Percentage'], 
               color=plt.cm.Reds(np.linspace(0.3, 0.9, len(accident_conditions))))
axes[0, 0].set_ylabel('Accident Percentage (%)', fontsize=11, fontweight='bold')
axes[0, 0].set_title('Pedestrian Accidents by Lighting Condition', fontsize=12, fontweight='bold')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(True, alpha=0.3, axis='y')

# Add annotations
for i, (cond, pct) in enumerate(zip(accident_conditions['Condition'], accident_conditions['Percentage'])):
    axes[0, 0].text(i, pct + 1, f"{pct}%", ha='center', fontweight='bold')

# 2. Risk factors
axes[0, 1].barh(accident_conditions['Condition'], accident_conditions['Risk_Factor'], 
                color=plt.cm.Oranges(np.linspace(0.3, 0.9, len(accident_conditions))))
axes[0, 1].set_xlabel('Relative Risk Factor', fontsize=11, fontweight='bold')
axes[0, 1].set_title('Detection Difficulty by Condition', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3, axis='x')

for i, (cond, risk) in enumerate(zip(accident_conditions['Condition'], accident_conditions['Risk_Factor'])):
    axes[0, 1].text(risk + 0.1, i, f"{risk}x", va='center', fontweight='bold')

# 3. Pedestrian types frequency
axes[1, 0].bar(range(len(pedestrian_types)), pedestrian_types['Frequency'], 
               color=plt.cm.Blues(np.linspace(0.3, 0.9, len(pedestrian_types))))
axes[1, 0].set_xticks(range(len(pedestrian_types)))
axes[1, 0].set_xticklabels(pedestrian_types['Type'], rotation=45, ha='right')
axes[1, 0].set_ylabel('Relative Frequency', fontsize=11, fontweight='bold')
axes[1, 0].set_title('Pedestrian Type Distribution', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# 4. Detection difficulty
axes[1, 1].scatter(pedestrian_types['Frequency'], pedestrian_types['Detection_Difficulty'], 
                  s=200, alpha=0.6, c=range(len(pedestrian_types)), cmap='viridis')
axes[1, 1].set_xlabel('Frequency', fontsize=11, fontweight='bold')
axes[1, 1].set_ylabel('Detection Difficulty', fontsize=11, fontweight='bold')
axes[1, 1].set_title('Frequency vs. Detection Difficulty', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

# Add labels
for i, row in pedestrian_types.iterrows():
    axes[1, 1].annotate(row['Type'], (row['Frequency'], row['Detection_Difficulty']),
                       fontsize=8, ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\n⚠️ Key Findings:")
print("   - 65%+ of pedestrian fatalities occur in low-light conditions")
print("   - Dark unlit roads: 7x higher risk than daylight")
print("   - Non-standard pedestrians (wheelchairs, objects) are hardest to detect")
print("\n💡 Challenge: Systems must work in ALL conditions, including rare ones!")

---

## 2️⃣ Load YOLOv8 Pedestrian Detector

We'll use YOLOv8 trained on COCO dataset (includes 'person' class).

In [ ]:
# Load YOLOv8 model
print("Loading YOLOv8n model...")
model = YOLO('yolov8n.pt')  # Nano version for speed

print("✅ Model loaded!")
print(f"\nModel classes include: {model.names[0]} (person)")
print(f"Total classes: {len(model.names)}")

# Hugging Face demo link
print("\n🤗 Try interactive demo:")
print("   https://huggingface.co/spaces/valid999/Yolov8_object_detection")
print("   Upload your own images to test pedestrian detection!")

---

## 3️⃣ Test on Various Pedestrian Scenarios

Let's test the model on different challenging scenarios.

In [ ]:
# Helper function to detect pedestrians
def detect_pedestrians(image, conf_threshold=0.25, visualize=True):
    """
    Detect pedestrians in image using YOLOv8.
    
    Args:
        image: numpy array (RGB)
        conf_threshold: minimum confidence
        visualize: whether to show results
    
    Returns:
        pedestrian_detections: list of detected pedestrians
    """
    # Run inference
    start_time = time.time()
    results = model.predict(image, conf=conf_threshold, verbose=False)[0]
    inference_time = (time.time() - start_time) * 1000
    
    # Filter for 'person' class (class_id = 0 in COCO)
    boxes = results.boxes.xyxy.cpu().numpy()
    confidences = results.boxes.conf.cpu().numpy()
    class_ids = results.boxes.cls.cpu().numpy().astype(int)
    
    pedestrians = []
    for box, conf, cls_id in zip(boxes, confidences, class_ids):
        if model.names[cls_id] == 'person':
            pedestrians.append({
                'bbox': box,
                'confidence': conf,
                'class': 'person'
            })
    
    if visualize:
        # Draw detections
        img_vis = image.copy()
        for ped in pedestrians:
            x1, y1, x2, y2 = ped['bbox'].astype(int)
            conf = ped['confidence']
            
            # Color based on confidence
            if conf > 0.7:
                color = (0, 255, 0)  # Green - high confidence
            elif conf > 0.5:
                color = (255, 165, 0)  # Orange - medium
            else:
                color = (255, 0, 0)  # Red - low confidence
            
            cv2.rectangle(img_vis, (x1, y1), (x2, y2), color, 3)
            cv2.putText(img_vis, f"Person: {conf:.2f}", (x1, y1-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # Display
        plt.figure(figsize=(14, 8))
        plt.imshow(img_vis)
        plt.title(f"Pedestrian Detection - {len(pedestrians)} detected | Inference: {inference_time:.1f}ms",
                 fontsize=14, fontweight='bold')
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    
    print(f"\n📊 Detection Summary:")
    print(f"   - Pedestrians detected: {len(pedestrians)}")
    print(f"   - Inference time: {inference_time:.1f} ms")
    print(f"   - FPS: {1000/inference_time:.1f}")
    
    if pedestrians:
        confidences_list = [p['confidence'] for p in pedestrians]
        print(f"   - Avg confidence: {np.mean(confidences_list):.3f}")
        print(f"   - Min confidence: {np.min(confidences_list):.3f}")
        print(f"   - Max confidence: {np.max(confidences_list):.3f}")
    
    return pedestrians

print("✅ Detection function ready!")

### Test on Sample Images

Let's create synthetic test scenarios representing different challenges.

In [ ]:
# Create synthetic pedestrian scenarios
def create_pedestrian_scene(scenario='normal', size=(1242, 375)):
    """
    Create synthetic pedestrian scene for testing.
    
    Scenarios:
    - normal: Clear daylight, standing pedestrians
    - occluded: Partially hidden pedestrians
    - night: Dark background
    - crowd: Multiple pedestrians close together
    """
    width, height = size
    image = np.zeros((height, width, 3), dtype=np.uint8)
    
    if scenario == 'normal':
        # Clear daylight scene
        image[:height//3] = [135, 206, 235]  # Sky
        image[height//3:] = [105, 105, 105]  # Road
        
        # Draw pedestrian silhouettes
        pedestrian_positions = [(300, 180), (600, 200), (900, 190)]
        for x, y in pedestrian_positions:
            # Simple person shape (head + body)
            cv2.circle(image, (x, y), 15, (200, 150, 100), -1)  # Head
            cv2.rectangle(image, (x-20, y+15), (x+20, y+80), (150, 100, 80), -1)  # Body
            cv2.rectangle(image, (x-20, y+80), (x-5, y+130), (100, 80, 60), -1)  # Left leg
            cv2.rectangle(image, (x+5, y+80), (x+20, y+130), (100, 80, 60), -1)  # Right leg
    
    elif scenario == 'occluded':
        # Partially occluded pedestrians
        image[:height//3] = [135, 206, 235]
        image[height//3:] = [105, 105, 105]
        
        # Draw cars (occluders)
        cv2.rectangle(image, (250, 200), (450, 320), (80, 80, 80), -1)
        cv2.rectangle(image, (800, 210), (1000, 330), (70, 70, 70), -1)
        
        # Partially visible pedestrians
        cv2.circle(image, (400, 180), 15, (200, 150, 100), -1)
        cv2.rectangle(image, (380, 195), (420, 250), (150, 100, 80), -1)
    
    elif scenario == 'night':
        # Dark scene
        image[:] = [20, 20, 30]  # Dark background
        
        # Dim pedestrians
        pedestrian_positions = [(400, 200), (700, 210)]
        for x, y in pedestrian_positions:
            cv2.circle(image, (x, y), 15, (80, 60, 50), -1)
            cv2.rectangle(image, (x-20, y+15), (x+20, y+80), (60, 50, 40), -1)
            cv2.rectangle(image, (x-20, y+80), (x-5, y+130), (50, 40, 30), -1)
            cv2.rectangle(image, (x+5, y+80), (x+20, y+130), (50, 40, 30), -1)
    
    elif scenario == 'crowd':
        # Multiple pedestrians
        image[:height//3] = [135, 206, 235]
        image[height//3:] = [105, 105, 105]
        
        pedestrian_positions = [(200, 180), (260, 190), (320, 185), 
                               (500, 200), (560, 195), (800, 190)]
        for x, y in pedestrian_positions:
            cv2.circle(image, (x, y), 12, (200, 150, 100), -1)
            cv2.rectangle(image, (x-15, y+12), (x+15, y+60), (150, 100, 80), -1)
            cv2.rectangle(image, (x-15, y+60), (x-5, y+100), (100, 80, 60), -1)
            cv2.rectangle(image, (x+5, y+60), (x+15, y+100), (100, 80, 60), -1)
    
    return image

# Test scenarios
scenarios = ['normal', 'occluded', 'night', 'crowd']

print("⚠️ Note: These are simplified synthetic images.")
print("   For real testing, use actual driving footage or public datasets.")
print("\nGenerating test scenarios...")

for scenario in scenarios:
    print(f"\n{'='*60}")
    print(f"Scenario: {scenario.upper()}")
    print(f"{'='*60}")
    
    img = create_pedestrian_scene(scenario)
    detections = detect_pedestrians(img, conf_threshold=0.25)
    
    # Analyze
    if scenario == 'normal' and len(detections) < 2:
        print("   ⚠️ False negatives likely!")
    elif scenario == 'occluded' and len(detections) < 1:
        print("   ⚠️ Failed to detect occluded pedestrians!")
    elif scenario == 'night' and len(detections) < 1:
        print("   ⚠️ Poor performance in low light!")

---

## 4️⃣ Failure Mode Analysis

Let's systematically analyze when pedestrian detection fails.

In [ ]:
# Common failure modes
failure_modes = pd.DataFrame({
    'Failure Mode': [
        'Occlusion',
        'Low Light / Night',
        'Motion Blur',
        'Small/Distant Objects',
        'Unusual Poses',
        'Weather (Rain/Fog)',
        'Crowded Scenes',
        'Non-standard Pedestrians'
    ],
    'Frequency': [30, 25, 15, 20, 10, 20, 15, 8],  # % of failures
    'Severity': [9, 10, 7, 8, 6, 9, 7, 10],  # 1-10 scale
    'Mitigation': [
        'Multi-view fusion, temporal tracking',
        'Thermal camera, LiDAR fusion',
        'Higher frame rate, prediction',
        'Higher resolution cameras, zoom',
        'More diverse training data',
        'Radar fusion, all-weather sensors',
        'Tracking, crowd-specific models',
        'Specialized training data'
    ]
})

display(failure_modes)

# Visualize failure modes
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Frequency
axes[0].barh(failure_modes['Failure Mode'], failure_modes['Frequency'],
            color=plt.cm.Reds(np.linspace(0.3, 0.9, len(failure_modes))))
axes[0].set_xlabel('Failure Frequency (%)', fontsize=12, fontweight='bold')
axes[0].set_title('Pedestrian Detection Failure Modes', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='x')

for i, (mode, freq) in enumerate(zip(failure_modes['Failure Mode'], failure_modes['Frequency'])):
    axes[0].text(freq + 0.5, i, f"{freq}%", va='center', fontsize=9)

# Severity vs Frequency
scatter = axes[1].scatter(failure_modes['Frequency'], failure_modes['Severity'],
                         s=300, alpha=0.6, c=range(len(failure_modes)), cmap='viridis')
axes[1].set_xlabel('Failure Frequency (%)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Severity (1-10)', fontsize=12, fontweight='bold')
axes[1].set_title('Failure Mode Risk Assessment', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Annotate high-risk quadrant
axes[1].axhline(y=7, color='red', linestyle='--', alpha=0.5)
axes[1].axvline(x=15, color='red', linestyle='--', alpha=0.5)
axes[1].text(22, 9.5, 'High Risk\n(Common + Severe)', 
            fontsize=11, color='red', fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Add labels
for i, row in failure_modes.iterrows():
    axes[1].annotate(row['Failure Mode'], 
                    (row['Frequency'], row['Severity']),
                    fontsize=7, ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\n🚨 High-Risk Failure Modes (Common + Severe):")
high_risk = failure_modes[(failure_modes['Frequency'] > 15) & (failure_modes['Severity'] > 7)]
for _, row in high_risk.iterrows():
    print(f"   - {row['Failure Mode']}: {row['Mitigation']}")

---

## 5️⃣ ISO 26262 Safety Requirements

**ISO 26262** is the automotive functional safety standard.

### ASIL Levels (Automotive Safety Integrity Level):
- **ASIL QM:** No safety requirement
- **ASIL A:** Lowest safety requirement
- **ASIL B:** Medium
- **ASIL C:** High
- **ASIL D:** Highest (safety-critical, e.g., braking, steering, pedestrian detection)

### Requirements for ASIL-D Pedestrian Detection:
1. **Redundancy:** Multiple sensors (camera + LiDAR + radar)
2. **Validation:** Extensive testing (millions of scenarios)
3. **Fail-safe:** Degraded mode if sensor fails
4. **Diagnostics:** Real-time sensor health monitoring
5. **Documentation:** Complete traceability from requirements to code
6. **Testing:** Hardware-in-the-loop (HIL), software-in-the-loop (SIL)
7. **Performance:** <100ms latency, >99.9% detection rate

In [ ]:
# ISO 26262 ASIL determination
# Based on: Severity, Exposure, Controllability

asil_table = pd.DataFrame({
    'Hazard': [
        'Pedestrian collision (high speed)',
        'Pedestrian collision (low speed)',
        'False positive (unnecessary brake)',
        'Delayed detection (>200ms)',
        'Complete sensor failure'
    ],
    'Severity': ['S3 (Fatal)', 'S2 (Severe injury)', 'S1 (Light injury)', 'S3 (Fatal)', 'S3 (Fatal)'],
    'Exposure': ['E4 (High)', 'E4 (High)', 'E3 (Medium)', 'E4 (High)', 'E2 (Low)'],
    'Controllability': ['C3 (Uncontrollable)', 'C2 (Difficult)', 'C1 (Simple)', 'C3 (Uncontrollable)', 'C3 (Uncontrollable)'],
    'ASIL': ['D', 'C', 'A', 'D', 'D']
})

display(asil_table)

print("\n💡 Key Takeaway:")
print("   - Pedestrian detection is ASIL-D (highest criticality)")
print("   - Failure can lead to FATAL consequences")
print("   - Requires extensive validation and redundancy")
print("\n🔗 ISO 26262 mandates:")
print("   - Sensor fusion (camera + LiDAR + radar)")
print("   - Real-time diagnostics")
print("   - Fail-operational or fail-safe design")
print("   - Extensive testing (>100M miles)")

---

## 6️⃣ Performance Metrics for Safety

For safety-critical applications, we care about more than just accuracy!

In [ ]:
# Define safety-critical metrics
safety_metrics = pd.DataFrame({
    'Metric': [
        'Recall (Detection Rate)',
        'Precision',
        'Latency',
        'False Negative Rate',
        'False Positive Rate',
        'Temporal Consistency',
        'Distance Detection Range',
        'Robustness (Edge Cases)'
    ],
    'Safety Requirement': [
        '>99.9%',
        '>95%',
        '<100ms',
        '<0.1%',
        '<5%',
        '>98% (frame-to-frame)',
        '>100m',
        '>90% (on long-tail scenarios)'
    ],
    'Why Critical': [
        'Missing pedestrian → collision',
        'Too many false alarms → driver distrust',
        'Late detection → no time to brake',
        'Direct safety impact',
        'Comfort & trust issue',
        'Tracking failures cause confusion',
        'Braking distance at highway speeds',
        'Real world has edge cases'
    ]
})

display(safety_metrics)

print("\n⚠️ Trade-offs:")
print("   - Higher recall (detect more) → More false positives")
print("   - Lower latency → May sacrifice accuracy")
print("   - Longer range → Lower resolution, harder detection")
print("\n💡 Safety-first approach: Prioritize RECALL over precision")
print("   Better to brake unnecessarily than miss a pedestrian!")

### Visualize Precision-Recall Trade-off

In [ ]:
# Simulate precision-recall curve
confidence_thresholds = np.linspace(0.1, 0.9, 20)

# Simulated data (in practice, computed from validation set)
recall = 1 - (confidence_thresholds - 0.1) / 0.8  # Higher threshold → lower recall
precision = 0.5 + 0.4 * (confidence_thresholds - 0.1) / 0.8  # Higher threshold → higher precision

# F1 score
f1 = 2 * precision * recall / (precision + recall)

# Plotting
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Precision-Recall curve
axes[0].plot(recall, precision, 'b-', linewidth=3, label='Detection Model')
axes[0].scatter(recall, precision, c=confidence_thresholds, cmap='viridis', s=100, zorder=5)
axes[0].axhline(y=0.95, color='green', linestyle='--', label='Precision Target (95%)', linewidth=2)
axes[0].axvline(x=0.999, color='red', linestyle='--', label='Recall Target (99.9%)', linewidth=2)
axes[0].set_xlabel('Recall (Detection Rate)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Precision', fontsize=12, fontweight='bold')
axes[0].set_title('Precision-Recall Trade-off', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].legend(fontsize=11)
axes[0].set_xlim(0.6, 1.0)
axes[0].set_ylim(0.4, 1.0)

# Add operating point
optimal_idx = np.argmax(f1)
axes[0].scatter(recall[optimal_idx], precision[optimal_idx], 
               s=300, c='red', marker='*', zorder=10, 
               label=f'Optimal (F1={f1[optimal_idx]:.3f})')

# Metrics vs threshold
axes[1].plot(confidence_thresholds, recall, 'b-', linewidth=2, label='Recall', marker='o')
axes[1].plot(confidence_thresholds, precision, 'g-', linewidth=2, label='Precision', marker='s')
axes[1].plot(confidence_thresholds, f1, 'r-', linewidth=2, label='F1-Score', marker='^')
axes[1].axhline(y=0.999, color='red', linestyle='--', alpha=0.5, label='Safety Target (99.9%)')
axes[1].set_xlabel('Confidence Threshold', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Score', fontsize=12, fontweight='bold')
axes[1].set_title('Metrics vs. Confidence Threshold', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].legend(fontsize=11)
axes[1].set_ylim(0.4, 1.0)

plt.tight_layout()
plt.show()

print("\n💡 For safety-critical systems:")
print("   - Choose LOW confidence threshold to maximize recall")
print("   - Accept some false positives (better safe than sorry)")
print("   - Use sensor fusion to reduce false positives")
print(f"\n   Optimal threshold for max F1: {confidence_thresholds[optimal_idx]:.2f}")
print(f"   But for safety, use lower threshold (~{confidence_thresholds[2]:.2f}) to ensure 99.9% recall")

---

## ✏️ Exercise: Safety Analysis

**Scenario:** You deployed a pedestrian detection system. After 1 million frames of testing:
- Total pedestrians: 50,000
- True Positives: 49,500
- False Positives: 1,000
- False Negatives: 500

**Questions:**
1. Calculate precision and recall
2. Is this system safe for ASIL-D?
3. What's the estimated risk?
4. What improvements would you recommend?

In [ ]:
# TODO: Calculate metrics and assess safety

# Given data
true_positives = 49500
false_positives = 1000
false_negatives = 500
total_pedestrians = 50000

# Calculate metrics
precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
f1_score = 2 * precision * recall / (precision + recall)
false_negative_rate = false_negatives / total_pedestrians

print("📊 System Performance:")
print(f"   Precision: {precision:.4f} ({precision*100:.2f}%)")
print(f"   Recall: {recall:.4f} ({recall*100:.2f}%)")
print(f"   F1-Score: {f1_score:.4f}")
print(f"   False Negative Rate: {false_negative_rate:.4f} ({false_negative_rate*100:.2f}%)")

print("\n⚠️ Safety Assessment:")
if recall >= 0.999:
    print("   ✅ Meets ASIL-D recall requirement (>99.9%)")
else:
    print(f"   ❌ FAILS ASIL-D requirement! (Recall: {recall*100:.2f}%, Need: >99.9%)")
    print(f"   ❌ Missing {false_negatives} pedestrians per 1M frames is UNACCEPTABLE")

if precision >= 0.95:
    print("   ✅ Meets precision requirement (>95%)")
else:
    print(f"   ⚠️ Precision below target ({precision*100:.2f}%)")

print("\n🚨 Risk Estimation:")
print(f"   - If vehicle drives at 30 fps video: {false_negatives * 30 / 1_000_000:.1f} missed pedestrians/sec")
print(f"   - At 50 km/h: ~14 m/s → {false_negatives * 14 / 1_000_000:.2f}m driven while missing pedestrian")
print(f"   - Risk: Collision if pedestrian within this distance")

print("\n💡 Recommended Improvements:")
print("   1. Lower confidence threshold to increase recall")
print("   2. Add LiDAR/radar fusion for redundancy")
print("   3. Collect more training data for edge cases")
print("   4. Implement temporal tracking (reduce single-frame misses)")
print("   5. Add fail-safe mechanisms (emergency braking)")

---

## 🎯 Key Takeaways

### Pedestrian Detection is Safety-Critical
- **ASIL-D classification** - highest automotive safety level
- **65%+ fatalities** occur in low-light conditions
- **Vulnerable road users** - no protection, unpredictable behavior

### Common Failure Modes
1. **Occlusion** - partially hidden pedestrians
2. **Low light / Night** - poor visibility
3. **Small/distant objects** - insufficient resolution
4. **Unusual poses** - outside training distribution
5. **Weather** - rain, fog degrade camera performance

### ISO 26262 Requirements
- **Redundancy:** Multiple sensors (camera + LiDAR + radar)
- **Performance:** >99.9% recall, <100ms latency
- **Validation:** Extensive testing (millions of scenarios)
- **Fail-safe:** Degraded mode if sensor fails
- **Documentation:** Complete traceability

### Safety-First Metrics
- **Prioritize RECALL** over precision (better to brake unnecessarily)
- **False negatives = potential collisions** (CRITICAL)
- **False positives = comfort issue** (annoying but not dangerous)
- **Temporal consistency** - track across frames

### Mitigation Strategies
1. **Sensor fusion** - combine camera + LiDAR + radar
2. **Lower confidence threshold** - detect more (accept some false positives)
3. **Temporal tracking** - smooth detections across frames
4. **Diverse training data** - include edge cases
5. **Thermal cameras** - for night/low-light
6. **Fail-safe mechanisms** - automatic emergency braking

### Real-World Lessons
- **Uber ATG (2018):** Classifier confusion + disabled emergency braking → FATAL
- **Tesla crashes:** Over-reliance on camera in low light
- **Industry standard:** Multi-sensor fusion is MANDATORY for Level 4+

---

## 🔜 Next Steps: Session 2

**Session 1 Complete!** You've learned:
- ✅ SAE automation levels and AV architecture
- ✅ Sensor modalities and fusion
- ✅ Object detection with deep learning
- ✅ Datasets and their limitations
- ✅ Pedestrian detection as safety-critical task

**Session 2 Preview:** AI Safety Failures & Adversarial Attacks
- Real accident case studies (Uber, Tesla, Cruise)
- Adversarial examples (physical patches that fool detectors)
- Out-of-distribution detection
- Uncertainty quantification
- Safety validation techniques

---

## 🤗 Interactive Demo

**Try YOLOv8 yourself:**

🔗 [Hugging Face YOLOv8 Demo](https://huggingface.co/spaces/valid999/Yolov8_object_detection)

**Exercise:**
1. Upload challenging pedestrian images:
   - Night scenes
   - Crowded streets
   - Unusual poses
   - Partial occlusions
2. Analyze failure cases
3. Think about mitigation strategies

---

*Notebook created by Milin Patel | Hochschule Kempten*  
*Last updated: 2025-01-17*